In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

In [11]:
flares = pd.read_excel('C:/Users/Raghav/Desktop/Solar Flare/Data/Solarflares.xlsx')
flares = flares.rename(columns = {'NOAA_Number' : 'NOAA_Number_flare'})
flares['Date'] = pd.to_datetime(flares['Date'])
flares['NOAA_Number_flare'] = flares['NOAA_Number_flare'].replace('     ', np.nan)
flares['NOAA_Number_flare'] = flares['NOAA_Number_flare'].replace('    ', np.nan)
flares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36522 entries, 0 to 36521
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   X_Ray_Class        36522 non-null  object        
 1   X_Ray_Intensity    36522 non-null  object        
 2   Integrated_Flux    35989 non-null  object        
 3   NOAA_Number_flare  19588 non-null  object        
 4   Date               36522 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 1.4+ MB


In [13]:
flares = flares[flares['X_Ray_Class'].isin(['X', 'M', 'C'])]
flares = flares[flares['NOAA_Number_flare'].notnull()]
flares['NOAA_Number_flare'] = flares['NOAA_Number_flare'].astype('int')

In [14]:
spots = pd.read_excel('C:/Users/Raghav/Desktop/Solar Flare/Data/sunspots.xlsx')
spots = spots.rename(columns = {'NOAA Number' : 'NOAA_Number_spots'})
spots['NOAA_Number_spots'] = spots['NOAA_Number_spots'].replace('          ', np.nan)
spots.dropna(inplace= True)
spots['Date'] = pd.to_datetime(spots['Date'])
spots['NOAA_Number_spots'] = spots['NOAA_Number_spots'].astype('object') 
spots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98138 entries, 0 to 98137
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Wilson Magnetic Classification  98138 non-null  object        
 1   NOAA_Number_spots               98138 non-null  object        
 2   Zurich Class                    98138 non-null  object        
 3   penumbra Class                  98138 non-null  object        
 4   Compactness Class               98138 non-null  object        
 5   Number of Spots                 98138 non-null  object        
 6   Longitudanal extent             98138 non-null  object        
 7   Area                            98138 non-null  object        
 8   Date                            98138 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 7.5+ MB


In [15]:
spots = spots.replace('     ' , np.nan)
spots = spots.dropna()
spots['NOAA_Number_spots'] = spots['NOAA_Number_spots'].astype('int')
spots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98137 entries, 0 to 98137
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Wilson Magnetic Classification  98137 non-null  object        
 1   NOAA_Number_spots               98137 non-null  int32         
 2   Zurich Class                    98137 non-null  object        
 3   penumbra Class                  98137 non-null  object        
 4   Compactness Class               98137 non-null  object        
 5   Number of Spots                 98137 non-null  object        
 6   Longitudanal extent             98137 non-null  object        
 7   Area                            98137 non-null  object        
 8   Date                            98137 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(7)
memory usage: 7.1+ MB


In [16]:
import datetime
from tqdm import tqdm

In [17]:
flare = []
for index,row in tqdm(flares.iterrows(),total=flares.shape[0]):
    noa_flare = row['NOAA_Number_flare']
    x_ray = row['X_Ray_Class']
    flare_date = row['Date']
    d1 ={}
    date_flare = row['Date']
    if x_ray == 'X' or x_ray == 'M' or x_ray == 'C':
      if noa_flare != np.nan:
        for i,r in spots.iterrows():
          noa_spot = r['NOAA_Number_spots']
          spot_date = r['Date']
          if noa_flare == noa_spot:
            diff_time = (flare_date - spot_date)/60
            a = []
            if diff_time < datetime.timedelta(minutes = 480):
              a = pd.concat([row,r],axis=0)
              a = a.to_dict()
              a["Status"] = "Associated"
              flare.append(a)
            break

associated_spots = pd.DataFrame(flare)

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [11:14<00:00,  4.75s/it]


In [18]:
associated_spots

,X_Ray_Class,X_Ray_Intensity,Integrated_Flux,NOAA_Number_flare,Date,Wilson Magnetic Classification,NOAA_Number_spots,Zurich Class,penumbra Class,Compactness Class,Number of Spots,Longitudanal extent,Area,Status
0,X,26,NaN,7978,1996-07-04 18:05:00,B,7978,B,X,O,2,2,10,Associated
1,X,21,5.6E-02,8100,1997-10-27 12:03:05,B,8100,C,S,O,5,10,60,Associated
2,X,94,3.6E-01,8100,1997-10-27 12:03:05,B,8100,C,S,O,5,10,60,Associated
3,X,26,7.0E-02,8113,1997-11-26 13:03:02,B,8113,E,K,O,5,13,200,Associated
4,X,10,1.6E-01,8210,1998-04-25 00:01:05,B,8210,C,S,O,3,8,180,Associated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,X,20,3.4E-01,12192,2014-10-17 01:00:05,B,12192,C,A,O,4,5,80,Associated
138,X,20,4.5E-01,12192,2014-10-17 01:00:05,B,12192,C,A,O,4,5,80,Associated
139,X,16,1.5E-01,12205,2014-11-04 00:03:05,A,12205,H,A,X,1,2,50,Associated
140,X,18,2.7E-01,12242,2014-12-14 16:00:03,BG,12242,D,A,I,10,4,100,Associated


In [19]:
#Create an outer join for the unassociated data points
a = (pd.merge(spots,flares, indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

In [20]:
a

,Wilson Magnetic Classification,NOAA_Number_spots,Zurich Class,penumbra Class,Compactness Class,Number of Spots,Longitudanal extent,Area,Date,X_Ray_Class,X_Ray_Intensity,Integrated_Flux,NOAA_Number_flare
0,B,7935.0,B,X,O,7,8,40,1995-12-26 12:04:09,NaN,NaN,NaN,NaN
1,B,7935.0,B,X,O,3,12,30,1995-12-26 15:03:06,NaN,NaN,NaN,NaN
2,B,7935.0,B,X,O,3,12,40,1995-12-26 19:05:00,NaN,NaN,NaN,NaN
3,B,7935.0,B,X,O,4,11,40,1995-12-27 00:00:01,NaN,NaN,NaN,NaN
4,B,7935.0,C,A,O,3,11,40,1995-12-27 14:01:07,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98132,A,12621.0,H,S,X,2,1,20,2016-12-28 07:04:00,NaN,NaN,NaN,NaN
98133,B,12621.0,B,X,O,3,6,10,2016-12-28 19:00:08,NaN,NaN,NaN,NaN
98134,A,12622.0,A,X,X,1,1,10,2016-12-30 01:00:00,NaN,NaN,NaN,NaN
98135,A,12622.0,A,X,X,1,0,10,2016-12-31 00:00:07,NaN,NaN,NaN,NaN


In [21]:
b= a.sort_values('Date', ascending=False).drop_duplicates(subset=['NOAA_Number_spots'])

In [22]:
sunspots_target = b.sort_values('Date')

In [23]:
sunspots_target = sunspots_target.reset_index()

In [25]:
sunspots_target['NOAA_Number_spots'] = sunspots_target['NOAA_Number_spots'].astype('int')

In [27]:
sunspots_target['Status'] = 'Unassociated'

In [28]:
sunspots_target

,index,Wilson Magnetic Classification,NOAA_Number_spots,Zurich Class,penumbra Class,Compactness Class,Number of Spots,Longitudanal extent,Area,Date,X_Ray_Class,X_Ray_Intensity,Integrated_Flux,NOAA_Number_flare,Status
0,10,B,7935,B,X,O,4,8,,1996-01-01 00:00:05,NaN,NaN,NaN,NaN,Unassociated
1,31,B,7937,B,X,O,3,3,10,1996-01-07 01:01:01,NaN,NaN,NaN,NaN,Unassociated
2,40,B,7938,C,A,O,6,8,90,1996-01-08 18:00:00,NaN,NaN,NaN,NaN,Unassociated
3,48,B,7940,B,X,O,2,2,10,1996-01-09 09:03:05,NaN,NaN,NaN,NaN,Unassociated
4,50,B,7939,B,X,O,3,2,20,1996-01-10 01:04:05,NaN,NaN,NaN,NaN,Unassociated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,98112,A,12618,A,X,X,1,0,10,2016-12-15 15:05:08,NaN,NaN,NaN,NaN,Unassociated
4421,98122,A,12619,A,X,X,1,0,0,2016-12-21 07:01:00,NaN,NaN,NaN,NaN,Unassociated
4422,98126,A,12620,H,S,X,1,1,10,2016-12-23 07:05:00,NaN,NaN,NaN,NaN,Unassociated
4423,98133,B,12621,B,X,O,3,6,10,2016-12-28 19:00:08,NaN,NaN,NaN,NaN,Unassociated


In [29]:
associated_spots['X_Ray_Class'].value_counts()

X    142
Name: X_Ray_Class, dtype: int64

In [32]:
associated_spots.to_excel("associated_spots_new.xlsx")

In [33]:
sunspots_target.to_excel('unassociated_spots_new.xlsx')